In [2]:
import os
import sys
from google.colab import files

repo = 'https://github.com/pol-ucd/SegmenterProject.git'
project = os.path.basename(repo).split('.')[0]

# Check CUDA GPUs are enabled
!nvidia-smi

if not os.path.exists(project):
    !git clone https://github.com/pol-ucd/SegmenterProject.git
else:
    os.chdir(project)
    !git pull https://github.com/pol-ucd/SegmenterProject.git


ROOT_DIR = os.path.abspath("")

os.chdir(project)

# To find local version of the library
sys.path.append(os.path.join(ROOT_DIR, 'SegmenterProject'))


Thu Aug  7 11:53:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
try:
  import torchinfo
  import torchmetrics
  import segmentation_models_pytorch
except:
  !pip install torchinfo
  !pip install torchmetrics
  !pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 865.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink

In [4]:
import torch
from torch import GradScaler

from nn.data import data_load
from nn.models import SegformerBinarySegmentation4
from nn.modules import CombinedLoss
from utils.torch_utils import TrainingManager, get_default_device




In [7]:
train_path = 'data/Polyp Segmentation/train'
val_path = 'data/Polyp Segmentation/valid'
n_epochs = 100
n_batch = 16
test_split = 0.3


# Set the default device to the best available GPU ... or CPU if no GPU available
device = get_default_device()
print(f"Using {device} device for model training.")

"""
I've implemented a data_load function that
can generate a train/test split if needed - but for now I'm just taking 100%
of the training and 100% validation data and using them to train and then to
validate respectively.
"""
(train_loader,
  _) = data_load(train_path,
                # test_split=args.test_split,
                test_split=0.0,  # Use 100% for training
                batch_size=n_batch,
                verbose=True)

(_,
  val_loader) = data_load(val_path,
                          # test_split=args.test_split,
                          test_split=1.0,  # Use 100% for testing/validation
                          batch_size=n_batch,
                          verbose=True)

n_val = len(val_loader) * n_batch
n_train = len(train_loader) * n_batch

print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(val_loader)}")

pretained_model = 'nvidia/segformer-b4-finetuned-ade-512-512'
# model = SegformerBinarySegmentation().to(device)  #Old Word doc model
model = SegformerBinarySegmentation4(pretrained_model=pretained_model,
                                     num_classes=1).to(device)
loss_fn = CombinedLoss()


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
                                                                 T_0=10,
                                                                 T_mult=2)

"""
Only use GradScaler if we have CUDA
"""
scaler = None
if torch.cuda.is_available():
    scaler = GradScaler()

trainer = TrainingManager(model,
                          optimizer,
                          criterion=loss_fn,
                          scaler=scaler,
                          train_loader=train_loader,
                          eval_loader=val_loader,
                          save_preds=False,
                          save_preds_path=""
                          )
train_params = {}
eval_params = {}
best_dice_score = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch + 1}/{n_epochs}")
    train_loss, train_dice = trainer.train(**train_params)
    print(f"Train Loss: {train_loss / n_train:.4f}, Train Dice: {train_dice / n_train:.4f}")

    val_loss, val_metrics = trainer.evaluate(**eval_params)
    print(
        f"Total evaluation Loss: {val_loss / n_val:.4f} | Dice: {val_metrics['dice'] / n_val:.4f} | IOU: {val_metrics['iou'] / n_val:.4f}")
    if val_metrics['dice'] > best_dice_score:
        best_dice_score = val_metrics['dice']
        torch.save(model.state_dict(), "best_segformer.pth")
        # _, _ = trainer.evaluate(save_preds=True)
        print(f"Model saved for dice score: {val_metrics['dice'] / n_val:.4f}")


Using cuda device for model training.
Found 630 training samples and 0 test samples
Found 0 training samples and 157 test samples
Training batches: 158
Test batches: 40


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/257M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

100%|██████████| 158/158 [01:12<00:00,  2.17it/s]

Train Loss: 0.0328, Train Dice: 0.1354


Total evaluation Loss: 0.0445 | Dice: 0.1144 | IOU: 0.0799
Epoch 4/100


 56%|█████▋    | 89/158 [00:41<00:32,  2.14it/s]


KeyboardInterrupt: 